In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimization
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from collections import Counter

In [8]:
Prediction = pd.read_csv('Prediction.csv')
data_taxi = pd.read_csv('Taxi_Data.csv')
data_uber = pd.read_csv('Uber-2015.csv')
data_taxi = data_taxi.drop(['Unnamed: 0'], axis = 1)
data_uber = data_uber.drop(['Unnamed: 0'], axis = 1)

In [35]:
Prediction_2014 = pd.read_csv('Prediction_2014.csv')

In [36]:
Prediction_2014.head()

,Location_ID,Uber_Popularity,Uber_a,Uber_b,Uber_c,Uber_d,Taxi_Popularity,taxi_a,taxi_b,taxi_c,taxi_d,Prediction
0,1,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,popular,1842.319261,3.909253,263.941479,-104.217617,Taxi_Dominates
1,2,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Not_Popular
2,3,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Not_Popular
3,4,popular,62.561609,0.580206,10.892951,-20.938528,popular,1176.796146,1.664393,-240.548271,-529.830022,Taxi_Safe
4,5,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Not_Popular


In [18]:
Prediction.head()

,Location_ID,Uber_Popularity,Uber_a,Uber_b,Uber_c,Uber_d,Taxi_Popularity,taxi_a,taxi_b,taxi_c,taxi_d,Prediction
0,1,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,popular,1701.463999,-0.103569,16.408720,236.862157,Taxi_Dominates
1,2,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Not_Popular
2,3,popular,10.167656,0.112903,4.818614,-1.116316,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Uber_Dominates
3,4,popular,395.943848,1.077022,141.060494,2.900637,popular,1106.532458,-0.670889,478.565018,-122.301229,225.5358995625786
4,5,Uber_not_popuplar,0.000000,0.000000,0.000000,0.000000,Taxi_not_popuplar,0.000000,0.000000,0.000000,0.000000,Not_Popular


In [34]:
## Plot real curv and regrrssion curv for location 4 and 161
def curv_func(x, a, b, c, d):
    return a + b*x + c*np.sin((2*np.pi/7.)*x) + d*np.cos((2*np.pi/7.)*x)

para_taxi = Prediction.loc[3, ['taxi_a', 'taxi_b', 'taxi_c', 'taxi_d']].tolist()
para_uber = Prediction.loc[3, ['Uber_a', 'Uber_b', 'Uber_c', 'Uber_d']].tolist()

pred_out_taxi = map(lambda x: curv_func(x, para_taxi[0], para_taxi[1], para_taxi[2], para_taxi[3]), range(len(count_value_taxi)))
pred_out_uber = map(lambda x: curv_func(x, para_uber[0], para_uber[1], para_uber[2], para_uber[3]), range(len(count_value_uber)))

pred_linear_out_taxi = map(lambda x: para_taxi[0]+para_taxi[1]*x, range(len(count_value_taxi)))
pred_linear_out_uber = map(lambda x: para_uber[0]+para_uber[1]*x, range(len(count_value_uber)))


# Create traces
trace_taxi_0 = go.Scatter(
    x = range(len(count_value_taxi)),
    y = count_value_taxi,
    mode = 'markers',
    name = 'Real Data Taxi'
)

trace_taxi_1 = go.Scatter(
    x = range(len(count_value_taxi)),
    y = pred_out_taxi,
    mode = 'lines',
    name = 'Regression Taxi'
)

trace_taxi_2 = go.Scatter(
    x = range(len(count_value_taxi)),
    y = pred_linear_out_taxi,
    name = 'Linear Taxi',
    line = dict(dash = 'dash') 
)

trace_uber_0 = go.Scatter(
    x = range(len(count_value_uber)),
    y = count_value_uber,
    mode = 'markers',
    name = 'Real Data Uber'
)

trace_uber_1 = go.Scatter(
    x = range(len(count_value_uber)),
    y = pred_out_uber,
    mode = 'lines',
    name = 'Regression Uber'
)

trace_uber_2 = go.Scatter(
    x = range(len(count_value_uber)),
    y = pred_linear_out_uber,
    name = 'Linear Uber',
    line = dict(dash = 'dash') 
)

data = [trace_taxi_0, trace_taxi_1, trace_taxi_2, trace_uber_0, trace_uber_1, trace_uber_2]
py.iplot(data, filename='Location 4')

In [40]:
Prediction.groupby(['Prediction'])['Location_ID'].count().loc[['Not_Popular', 'Taxi_Dominates', 'Taxi_Safe', 'Uber_Dominates']]

Prediction
Not_Popular       40
Taxi_Dominates    10
Taxi_Safe         12
Uber_Dominates    86
Name: Location_ID, dtype: int64

In [41]:
Prediction_2014.groupby(['Prediction'])['Location_ID'].count().loc[['Not_Popular', 'Taxi_Dominates', 'Taxi_Safe', 'Uber_Dominates']]

Prediction
Not_Popular       116
Taxi_Dominates     36
Taxi_Safe          54
Uber_Dominates      7
Name: Location_ID, dtype: int64

In [116]:
Classification_2015 = np.zeros((len(Prediction)), dtype=object)
for i in range(len(Prediction)):
    if Prediction.loc[i, 'Prediction'] == 'Not_Popular':
        Classification_2015[i] = 'Low Trips'
    elif Prediction.loc[i, 'Prediction'] == 'Taxi_Dominates' or Prediction.loc[i, 'Prediction'] == 'Taxi_Safe':
        Classification_2015[i] = 'Taxi Dominates'
    elif Prediction.loc[i, 'Prediction'] == 'Uber_Dominates':
        Classification_2015[i] = 'Uber Dominates'
    elif Prediction.loc[i, 'Prediction'] < 0:
        Classification_2015[i] = 'Uber Dominates'
    else:
        Classification_2015[i] = 'Expected Uber Dominates'

In [117]:
Classification_2014 = np.zeros((len(Prediction_2014)), dtype=object)
for i in range(len(Prediction_2014)):
    if Prediction_2014.loc[i, 'Prediction'] == 'Not_Popular':
        Classification_2014[i] = 'Low Trips'
    elif Prediction_2014.loc[i, 'Prediction'] == 'Taxi_Dominates' or Prediction_2014.loc[i, 'Prediction'] == 'Taxi_Safe':
        Classification_2014[i] = 'Taxi Dominates'
    elif Prediction_2014.loc[i, 'Prediction'] == 'Uber_Dominates':
        Classification_2014[i] = 'Uber Dominates'
    elif Prediction_2014.loc[i, 'Prediction'] < 0:
        Classification_2014[i] = 'Uber Dominates'
    else:
        Classification_2014[i] = 'Expected Uber Dominates'

In [118]:
class_2015 = Counter(Classification_2015)

In [119]:
class_2014 = Counter(Classification_2014)

In [120]:
trace_2014 = go.Bar(
    x=['Low Trips', 'Taxi Dominates', 'Uber Dominates', 'Expected Uber Dominates'],
    y=[class_2014['Low Trips'], class_2014['Taxi Dominates'], class_2014['Uber Dominates'], class_2014['Expected Uber Dominates']],
    name='2014'
)

trace_2015 = go.Bar(
    x=['Low Trips', 'Taxi Dominates', 'Uber Dominates', 'Expected Uber Dominates'],
    y=[class_2015['Low Trips'], class_2015['Taxi Dominates'], class_2015['Uber Dominates'], class_2015['Expected Uber Dominates']],
    name='2015'
)

data = [trace_2014, trace_2015]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='Taxi vs Uber')

In [122]:
Prediction['Prediction_Type'] = Classification_2015

In [123]:
Prediction_2014['Prediction_Type'] = Classification_2014

In [126]:
Prediction.to_csv('Prediction_2015.csv')

In [238]:
taxi_2011_06 = data_taxi[(data_taxi['Year'] == 2011) & (data_taxi['Month'] == 6)]

In [239]:
taxi_2011_06 = taxi_2011_06.groupby(['Location_ID'])['Count'].sum()

In [240]:
taxi_2015_06 = data_taxi[(data_taxi['Year'] == 2015) & (data_taxi['Month'] == 6)]
taxi_2015_06 = taxi_2015_06.groupby(['Location_ID'])['Count'].sum()

In [296]:
uber_2015_06 = data_uber[data_uber['Month'] == 6]

In [298]:
uber_2015_06 = uber_2015_06.groupby('locationID')['time'].count()

In [300]:
uber_2015_06 = uber_2015_06.reset_index(level=0)

In [304]:
result_1 = uber_2015_06.rename(columns={'locationID': 'Location_ID'})

In [245]:
taxi_2011_06 = taxi_2011_06.reset_index(level=0)
taxi_2015_06 = taxi_2015_06.reset_index(level=0)

In [246]:
taxi_summary = pd.merge(taxi_2011_06, taxi_2015_06, on = ['Location_ID'])

In [306]:
result = pd.merge(uber_2015_06, taxi_summary, on = ['Location_ID'], how='left')

In [307]:
result = result.fillna(0)

In [310]:
result.rename(columns={'time': 'Count', 'Count_x': 'Count_2011', 'Count_y':'Count_2015'}, inplace=True)

In [312]:
result['sum'] = result['Count']+result['Count_2015']

In [324]:
result['summary'] = map(int, result['sum']>result['Count_2011']*1.5)

In [328]:
result['Transformed_ID'] = map(ID_Transform, result.loc[:, 'Location_ID'].tolist())

In [330]:
result.to_csv('Uber_Taxi_Summary.csv', index=False)

In [212]:
summary = pd.read_csv('summary.csv')

In [215]:
summary['uber+taxi'] = summary['Count_uber_2015']+summary['Count_2015']

In [221]:
summary[summary['uber+taxi']>summary['Count_2009']*2]

,Location_ID,Count_uber_2015,Count_2009,Count_2015,uber+taxi
2,3,860,94,41,901
8,9,647,195,25,672
13,14,7586,1091,584,8170
14,15,685,207,27,712
15,16,1964,136,39,2003
16,17,17455,816,1383,18838
17,18,1449,99,93,1542
19,20,1692,82,42,1734
20,21,1870,931,119,1989
21,22,2245,0,0,2245


In [334]:
data_taxi_sample_2 = data_taxi[(data_taxi['Year'] == 2015) & (data_taxi['Month']<=6)]

In [336]:
taxi_result = data_taxi_sample_2.groupby('Location_ID')['Count'].sum()

In [337]:
taxi_result = taxi_result.reset_index(level=0)

In [340]:
taxi_result.drop(0, axis = 0, inplace=True)

In [342]:
taxi_result['Location_ID_new'] = map(ID_Transform, taxi_result.loc[:, 'Location_ID'].tolist())

C:\Users\Hao\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning:

using a non-integer number instead of an integer will result in an error in the future



In [344]:
taxi_result.to_csv('Taxi_distribution.csv')

In [190]:
Location_transform = np.load('nyc_lookup.npy')

In [196]:
def ID_Transform(id_value, trans = Location_transform):
    index = trans[id_value-1][-1]
    return index

result = map(ID_Transform, data_uber.loc[:, 'locationID'].tolist())

In [198]:
data_uber.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,Date,Year,Month,time
0,B02617,2015-05-17 09:47:00,B02617,141,17,2015,5,9
1,B02617,2015-05-17 09:47:00,B02617,65,17,2015,5,9
2,B02617,2015-05-17 09:47:00,B02617,100,17,2015,5,9
3,B02617,2015-05-17 09:47:00,B02774,80,17,2015,5,9
4,B02617,2015-05-17 09:47:00,B02617,90,17,2015,5,9


In [203]:
data_uber['Transformed_ID'] = result

In [206]:
uber_distribution = data_uber.groupby(['Transformed_ID'])['time'].count().reset_index(level=0)

In [208]:
uber_distribution.rename(columns={'Transformed_ID': 'Location_ID_new', 'time': 'Count'}, inplace=True)

In [210]:
uber_distribution.to_csv('uber_distribution.csv')

In [189]:
result.to_csv('summary.csv', index = False)

In [178]:
uber_2015_06.rename(columns={'locationID': 'Location_ID', 'time': 'Count_uber_2015'}, inplace=True)